In [4]:
!nvidia-smi

Fri Apr 24 10:46:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.116.00   Driver Version: 418.116.00   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   38C    P0    37W / 300W |      0MiB / 16130MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A   40C    P0    36W / 300W |      0MiB / 16130MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
import os
import sys
sys.path.append('..')

import numpy as np
import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

Num GPUs Available:  4


In [6]:
tf.__version__

'2.2.0-dev20200418'

In [7]:
from bert.model import create_albert_model
from bert.losses import ECE, masked_sparse_categorical_crossentropy
from bert.optimization import create_optimizer

opt = create_optimizer(1E-4, 500, 10000)

strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")

with strategy.scope():
    model = create_albert_model(model_dimension=768,
                                transformer_dimension=768 * 4,
                                num_attention_heads=768 // 64,
                                num_transformer_layers=12,
                                vocab_size=24,
                                dropout_rate=0.,
                                max_relative_position=128)
    
    model.compile(
        loss=masked_sparse_categorical_crossentropy,
        metrics=[ECE],
        optimizer=opt,
        experimental_run_tf_function=True)    

    
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 768)         18432     
_________________________________________________________________
transformer (Transformer)    (None, None, 768)         7104320   
_________________________________________________________________
transformer_1 (Transformer)  (None, None, 768)         7104320   
_________________________________________________________________
transformer_2 (Transformer)  (None, None, 768)         7104320   
_________________________________________________________________
transformer_3 (Transformer)  (None, None, 768)         7104320   
_________________________________________________________________
transformer_4 (Transformer)  (None, None, 768)         710432

In [14]:
import os
from bert.dataset import create_masked_input_dataset
data_path = '/gpfs/alpine/proj-shared/bie108/split_uniref100'

train_data_dir = os.path.join(data_path, 'train_uniref100_split')
train_data_files = [os.path.join(train_data_dir, f) for f in os.listdir(train_data_dir)]

valid_data_dir = os.path.join(data_path, 'dev_uniref50_split')
valid_data_files = [os.path.join(valid_data_dir, f) for f in os.listdir(valid_data_dir)]

with tf.device('/CPU:0'):
    training_data = create_masked_input_dataset(
        sequence_path=train_data_files,
        max_sequence_length=512,
        fix_sequence_length=True,
        batch_size=16)
    
    training_data = training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)    

    valid_data = create_masked_input_dataset(
        sequence_path=valid_data_files,
        max_sequence_length=512,
        batch_size=16)

    valid_data = valid_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [15]:
print(next(iter(training_data)))

(<tf.Tensor: shape=(16, 512), dtype=int32, numpy=
array([[ 2, 14,  1, ...,  0,  0,  0],
       [13, 19, 11, ..., 13, 16, 19],
       [ 2,  1, 16, ...,  0,  0,  0],
       ...,
       [ 2, 14,  6, ...,  0,  0,  0],
       [ 2, 14,  6, ...,  0,  0,  0],
       [ 2, 14,  1, ...,  0,  0,  0]], dtype=int32)>, <tf.Tensor: shape=(16, 512), dtype=int32, numpy=
array([[ 0,  0, 12, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0, 11,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  3, ...,  0,  0,  0]], dtype=int32)>)


In [13]:
model.fit(training_data, steps_per_epoch=50, epochs=5,
          verbose=1, validation_data=valid_data, validation_steps=10)

Epoch 1/5


ResourceExhaustedError:  OOM when allocating tensor with shape[24,12,512,512] and type half on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/transformer_11/relative_attention_11/MatMul (defined at ../bert/attention_utils.py:41) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_57046]

Errors may have originated from an input operation.
Input Source operations connected to node model/transformer_11/relative_attention_11/MatMul:
 model/transformer_11/relative_attention_11/transpose_1 (defined at ../bert/layers.py:78)

Function call stack:
train_function


In [ ]:
390 / 8

In [ ]:
552 / 12

In [ ]:
masked, true = next(iter(training_data))
predictions = model.predict(masked)

In [ ]:
masked[:, 0]